In [1]:
%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


In [2]:
N_files = 10
iopt = 11

In [7]:
N_centr = 5
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
central_bins = [0,20,20,40,40,60,60,80,80,93]
pt_boarders = [0.5,0.9,0.2,0.5,0.5,1.0,1.0,5.0,0.5,5]
file_path="/media/yoren/T7 Shield/PHENIX/Dielectrons/fulls/mass_20095/"
file_names=["411601.root"] 
sim_file_path="../sim/output/Newembed/sngl/"
legend_names = ["real","#pi^{0}","#eta","#gamma #rightarrow ee","c#bar{c}","b#bar{b}"] 
hist_select_3D_names = [f"inv_mass_ee_DCA_V{iopt}_FG12",f"inv_mass_ee_DCA_V{0}_FG12"]

In [8]:
for i  in range(len(hist_select_3D_names)):
    hist_select_3D_names[i] = hist_select_3D_names[i]

In [9]:
N_hists = len(hist_select_3D_names)
loc = True
Save_to_html = False

In [10]:
hists_read, hists_sim = [], []

for iFile in range(len(file_names)):
    print(file_path+file_names[iFile])
    infile = root.TFile.Open(file_path+file_names[iFile], "read")
    hists_read_file = []
    for ihist in range(N_hists):
        hists_read_file.append(infile.Get("c00_z00_r00/" + hist_select_3D_names[ihist]))
        hists_read_file[-1].SetDirectory(root.nullptr)
        hists_read_file[-1].SetName(hists_read_file[-1].GetName()+f"{iFile}")
    hists_read.append(hists_read_file)


infile.Close()

/media/yoren/T7 Shield/PHENIX/Dielectrons/fulls/mass_20095/411601.root


In [11]:
type_loc = 0
pt = [1.35,2.45]
c0 = root.TCanvas(f"c0",f"c0",700,700)
c0.Divide(1,1)
legends = []
proj_y = hists_read[0][type_loc].ProjectionY(hists_read[0][type_loc].GetName()+f"pos_proj_y",1,-1,1,-1)
biny0 = proj_y.FindBin(pt[0])
biny1 = proj_y.FindBin(pt[1])
for isec in range(1):
    c0.cd(1+isec)
    ibinz0 = 16
    ibinz1 = 17
    pos_proj = hists_read[0][type_loc].ProjectionX(hists_read[0][type_loc].GetName()+f"pos_proj_{isec}",biny0,biny1,ibinz0,ibinz1)
    Format_Hist_total(pos_proj,title_x="x, cm",offset_y=1.1)
    root.gPad.SetLogy()
    pos_proj.Draw()
    #pos_proj.Fit("gaus","R","")
    legends.append(root.TLegend(0.5,0.6,0.95,0.95,f"Sector: {isec}, pT={pt[0]}"))
    legends[-1].AddEntry(pos_proj,legend_names[0],"l")
    Format_Legend(legends[-1])
    legends[-1].Draw()
    #print(pos_proj.Integral(pos_proj.FindBin(0.27),pos_proj.FindBin(0.4))/pos_proj.Integral())
    print(f"Mean and Standard deviation for sec {isec}: {pos_proj.GetMean()*10000:.0f}, {pos_proj.GetStdDev()*10000:.0f} mum")

if not Save_to_html: c0.Draw()
if not Save_to_html: c0.SaveAs(f"output/rich_emcal_qa/qa_pt_sec.png")

Mean and Standard deviation for sec 0: 6820, 4128 mum


python ERROR: cannot open image file "output/rich_emcal_qa/qa_pt_sec.png" for writing. Please check permissions.


reading file list

In [12]:
import os
import re

file_pattern = re.compile(r"(\d{6})\.root")

file_numbers = []

for filename in os.listdir(file_path):
    match = file_pattern.match(filename)
    if match:
        file_numbers.append(match.group(1))

print(len(file_numbers))

1071


In [13]:
file_numbers.sort()

In [14]:
#file_numbers= file_numbers[:N_files]

In [15]:
pt = [0.45,4.45]
biny0 = proj_y.FindBin(pt[0])
biny1 = proj_y.FindBin(pt[1])

In [29]:
def vertex_estimator(file_number):
    infile = root.TFile.Open(file_path+f"{file_number}.root", "read")
    hists_read_file = []
    hits_stat = infile.Get("c00_z00_r00/PoolStatistics")
    hits_stat.SetDirectory(root.nullptr)

    for ihist in range(N_hists):
        hists_read_file.append(infile.Get("c00_z00_r00/" + hist_select_3D_names[ihist]))
        hists_read_file[-1].SetDirectory(root.nullptr)
        hists_read_file[-1].SetName(hists_read_file[-1].GetName()+f"{file_number}")
        for ic in range(1,4):
            hists_read_file[-1].Add(infile.Get(f"c0{ic}_z00_r00/" + hist_select_3D_names[ihist]))
    
    for ic in range(1,4):
        hits_stat.Add(infile.Get(f"c0{ic}_z00_r00/PoolStatistics"))
            
    params_to_return = []
    for ihist in range(N_hists):
        c1 = root.TCanvas(f"c1_{file_number}_{ihist}", f"c1_{file_number}_{ihist}", 600, 600)
        pos_projX = hists_read_file[ihist].ProjectionX(hists_read_file[ihist].GetName()+f"pos_projX_{file_number}_{ihist}",1,40,1,12)
        integral = pos_projX.Integral(biny0,biny1)
        int_275_290 = pos_projX.Integral(pos_projX.FindBin(2.755),pos_projX.FindBin(2.94))
        int300_320 = pos_projX.Integral(pos_projX.FindBin(3.00),pos_projX.FindBin(3.40))

        entries = hits_stat.GetBinContent(2)
        params_to_return.append((integral, int_275_290, int300_320, entries))

    return params_to_return



In [30]:
#file_numbers = file_numbers[:10]
#print(file_numbers)

In [31]:
int0, int1, int2, entries = [[] for _ in range(N_hists)], [[] for _ in range(N_hists)], [[] for _ in range(N_hists)], [[] for _ in range(N_hists)]
for file_number in file_numbers:
    params_for_all_secotrs = vertex_estimator(file_number)
    for isec in range(N_hists):
        mean, sigma, mean_err, entries_local = params_for_all_secotrs[isec]
        int0[isec].append(mean)
        int1[isec].append(sigma)
        int2[isec].append(mean_err)   
        entries[isec].append(entries_local)

In [32]:
print(int2)

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 2.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0

In [33]:
int3 = [ [x / y if  y !=0 else 1 for x, y in zip(int2[isec], int1[isec])] for isec in range(N_hists)]
print(int3)

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.0, 1, 1, 1, 1, 1, 1, 1, 1, 0.0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2.0, 1, 1, 1, 1, 0.0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 0.0, 1, 1, 1, 1, 0.0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1, 0.0, 1, 1, 1, 0.0, 0.0, 1, 1, 1, 1, 1, 1, 1, 0.0, 0.0, 1, 1, 1, 1, 1, 0.0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.0, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1, 1, 1, 1, 1, 1.0, 1, 1, 1, 0.0, 1, 1, 1, 2.0, 1, 1, 1, 1, 1, 1, 1, 0.0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.0, 1, 1, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1, 0.5, 1, 1.0, 1, 0.0, 0.0, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 1, 0.0, 1, 1, 1, 1, 1.0, 1, 0.0, 0.0, 1, 0.0, 2.0, 1, 0.0, 1, 0.0, 1, 0.0, 1, 1, 1, 1, 1, 0.0, 0.0, 2.0, 1, 1, 1, 1, 1, 0.0, 1, 1, 1, 0.0, 1, 1, 1, 1, 0.0, 0.0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.0, 2.0, 1, 1, 1, 1, 1, 1, 0.0, 0.0, 1, 1, 0.0, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [34]:
# Create canvases
c1 = root.TCanvas("c1", "Mean  vs File Numbers", 1200,600)

# Convert file_numbers to integers for plotting
file_numbers_int = [int(num) for num in file_numbers]

# Create graphs

# Customize and draw graphs
def customize_and_draw(iOpt, N_hists, canvas, title, x_title, y_title):

    canvas.Divide(2)
    graphs = []
    for ihist in range(N_hists):
        canvas.cd(ihist+1)
        graph =  root.TGraphErrors(len(file_numbers), array('d', file_numbers_int),  array('d', int3[ihist]), 0, 0)
        graph.SetTitle(title)
        Format_Hist_total(graph, Msize=1, Mcolor=4, Mstyle=20, title_x=x_title, title_y=y_title, left=0.17, offset_y=1.1)
        graph.Draw("AP")
        graph.GetYaxis().SetRangeUser(0,5)
        canvas.Draw()
        graphs.append(graph)

    return graphs

results = []
results.append(customize_and_draw(0,2, c1, "Mean vs File Numbers", "File Numbers", "Mean"))

In [35]:
print(int(file_numbers[1000]))

414559


In [36]:
good_run_list = []; bad_run_list = []
for irun in range(len(file_numbers)):
    if int3[0][irun] > 0.3 or int3[1][irun] > 0.3 or int(file_numbers[irun]) < 411000:
        good_run_list.append(file_numbers[irun])
    else:
        bad_run_list.append(file_numbers[irun])

In [37]:
print(len(good_run_list), len(bad_run_list))

1035 36


In [38]:
print(bad_run_list)

['411008', '411152', '411354', '411561', '411763', '411819', '411926', '412068', '412328', '412879', '413077', '413079', '413088', '413187', '413242', '413539', '413561', '413612', '413662', '413747', '413931', '414061', '414136', '414148', '414152', '414207', '414331', '414606', '414626', '414634', '414702', '414728', '414825', '414829', '414886', '414986']


In [ ]:
print("hadd -f -k -O -j 16 -n 10000 ../m_ee_Run14AuAu_160th_new_20095_1071runs_goodruns1.root " + " ".join([f"{run}.root " for run in good_run_list]))

hadd -f -k -O -j 16 -n 10000 ../m_ee_Run14AuAu_157th_new_20079_goodruns1.root 405863.root  405864.root  405865.root  405868.root  405869.root  405961.root  405962.root  405964.root  405966.root  405971.root  405972.root  405973.root  405975.root  405995.root  405996.root  406087.root  406088.root  406089.root  406090.root  406093.root  406094.root  406095.root  406099.root  406180.root  406182.root  406183.root  406190.root  406257.root  406258.root  406259.root  406261.root  406263.root  406265.root  406266.root  406268.root  406539.root  406540.root  406543.root  406544.root  406546.root  406549.root  406555.root  406556.root  406579.root  406581.root  406582.root  406584.root  406661.root  406662.root  406663.root  406666.root  406671.root  406674.root  406675.root  406676.root  406677.root  406696.root  406697.root  406698.root  406700.root  406745.root  406746.root  406747.root  406748.root  406753.root  406754.root  406759.root  406760.root  406761.root  406762.root  406764.root 